In [3]:
import warnings
warnings.filterwarnings('ignore')

import os
from glob import glob
import pickle

import numpy as np
import pandas as pd

import nltk
import re
import codecs
import unidecode
#pip install unidecode
import mpld3
# pip install mpld3
import stop_words
# pip install stop-words
from nltk import SnowballStemmer, pos_tag, word_tokenize, wordpunct_tokenize
from nltk.corpus import stopwords

from sklearn.utils import shuffle
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import HashingVectorizer,TfidfTransformer,TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import AgglomerativeClustering
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn.preprocessing import Normalizer
from sklearn.metrics.pairwise import *
from sklearn.linear_model import LogisticRegression
from sklearn.svm import *
from sklearn.semi_supervised import *

** Lecture des données **

In [4]:
def load_cv_list(nombre):
    path = '../data_indeed/Txt/'
    liste_paths = [path+directory for directory in os.listdir(path)]
    liste_cv = []
    liste_files = []
    for path in liste_paths :
        if "informaticien" and "dba" and "chef_de_projet_informatique" not in path:
            filenames = sorted(glob(os.path.join(path,"*.txt")))
           
            for file in filenames[:nombre]:
                liste_cv.append(open(file).read())
                liste_files.append(file.split('/')[-1].split('.')[0])
    return liste_cv,liste_files

In [5]:
liste_cv_indeed,liste_files = load_cv_list(200)
len(liste_cv_indeed)

2000

### Cleaning fonctions

#### Suppression des sauts de ligne

In [6]:
import string,re

In [7]:
regex = re.compile('[%s]' % '(\\n)*(\\x0c)*')
def del_line_feed(s):  
    """Delete \n in the text"""
    return regex.sub(' ', s)

In [8]:
liste_cv_indeed = [del_line_feed(text).lower() for text in liste_cv_indeed]
liste_cv_indeed[0]

"informaticien développement et réseaux  développeur intégrateur web  éragny  95  - email me on indeed: indeed.com/r/d7e8913ed00d0384  aujourd’hui, je suis en recherche d'une opportunité sur un poste de développeur ou d’intégrateur web afin de monter toujours plus en compétence et d’approfondir les bases solide que j’ai acquis en formation.  expérience  informaticien développement et réseaux  legrandcercle95  -  éragny  95  -  novembre 2016 - juin 2017  informaticien de l'entreprise, mes missions était de gérer les différent problème dans l'entreprise. mise en place d'un antivirus serveur, sauvegarde nas... créé et gérer les droits sur l'active directory. paramétrer des clients léger ainsi que du matériel informatique comme des imprimantes ou des étiqueteuse en ip fixe.  réajustement du code html et css sur le site grand public selon les normes w3c. création d'une source odbc création d'un code en php - sql afin de récupéré des données librairie sur un site fournisseur pour les enregis

#### Suppression ponctuation

In [9]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [10]:
#le maintient de la ponctuation pertube le stop words, apostrophe gérée dans text_treatment
regex = re.compile('[%s]' % re.escape('!"#$%&\()*+,-./:;<=>?@[\\]^_{|}~')) 

def del_punct(s):  
    """Delete punctuation in the text"""
    return regex.sub('', s)

#test 
liste_cv_indeed_no_punc = [del_punct(text) for text in liste_cv_indeed]

liste_cv_indeed_no_punc[0]

"informaticien développement et réseaux  développeur intégrateur web  éragny  95   email me on indeed indeedcomrd7e8913ed00d0384  aujourd’hui je suis en recherche d'une opportunité sur un poste de développeur ou d’intégrateur web afin de monter toujours plus en compétence et d’approfondir les bases solide que j’ai acquis en formation  expérience  informaticien développement et réseaux  legrandcercle95    éragny  95    novembre 2016  juin 2017  informaticien de l'entreprise mes missions était de gérer les différent problème dans l'entreprise mise en place d'un antivirus serveur sauvegarde nas créé et gérer les droits sur l'active directory paramétrer des clients léger ainsi que du matériel informatique comme des imprimantes ou des étiqueteuse en ip fixe  réajustement du code html et css sur le site grand public selon les normes w3c création d'une source odbc création d'un code en php  sql afin de récupéré des données librairie sur un site fournisseur pour les enregistré en fiche xml cré

** Reconnaissance du langage du CV**

In [11]:
def _calculate_languages_ratios(text):
    """
    Calculate probability of given text to be written in several languages and
    return a dictionary that looks like {'french': 2, 'spanish': 4, 'english': 0}
    """

    languages_ratios = {}

    '''
    nltk.wordpunct_tokenize() splits all punctuations into separate tokens
    
    >>> wordpunct_tokenize("That's thirty minutes away. I'll be there in ten.")
    ['That', "'", 's', 'thirty', 'minutes', 'away', '.', 'I', "'", 'll', 'be', 'there', 'in', 'ten', '.']
    '''

    tokens = wordpunct_tokenize(text)
    words = [word.lower() for word in tokens] #from text get list of word in minuscule

    
    for language in stopwords.fileids(): # pour chaque langue
        stopwords_set = set(stopwords.words(language)) #je mets les stop words du langage dans un set
        words_set = set(words) #je mets les mots de mon texte dans un set
        #je prends l'intersection entre les mots de mon texte et les mots du stopwords dans le langage donné
        common_elements = words_set & stopwords_set
        
        #je compute mon score comme le nombre d'éléments en communs dictionnaire [langage : score]
        languages_ratios[language] = len(common_elements) # language "score"

    return languages_ratios

In [12]:
import nltk
nltk.download('stopwords')
stopwords.fileids()

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/mehdiregina/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['arabic',
 'azerbaijani',
 'danish',
 'dutch',
 'english',
 'finnish',
 'french',
 'german',
 'greek',
 'hungarian',
 'indonesian',
 'italian',
 'kazakh',
 'nepali',
 'norwegian',
 'portuguese',
 'romanian',
 'russian',
 'spanish',
 'swedish',
 'turkish']

In [13]:
def get_cv_langue(liste_cv, language,cv_names) :
    """Return resume witten in the specified language in parameter"""
    liste_2 = []
    french_cv_names = []
    i=0
    for cv in liste_cv:
        if max(_calculate_languages_ratios(cv),key =_calculate_languages_ratios(cv).get)=='french':
            liste_2.append(cv)
            french_cv_names.append(cv_names[i])
        i+=1
    return liste_2,french_cv_names

In [14]:
liste_cv_indeed_fr, liste_files_fr = get_cv_langue(liste_cv_indeed_no_punc,'french',liste_files)
nb_cv = len(liste_cv_indeed_no_punc)
nb_cv_fr = len(liste_cv_indeed_fr)

print("proportion cv french :",1- ((nb_cv-nb_cv_fr)/nb_cv))


proportion cv french : 0.704


In [15]:
len(liste_cv_indeed_fr)

1408

** Preprocessing du text **

In [16]:
def text_treatment (text):
    text = text.lower()
    text = text.replace("\x00", '').replace("\x01", '').replace("\x02", '').replace("\x03", '') \
    .replace("\x04", '').replace("\x05", '').replace("\x06", '').replace("\x07", '').replace("\x08", '') \
    .replace("\x0e", '').replace("\x11", '').replace("\x12", '').replace("\x10", '').replace("\x19", '') \
    .replace("\x1b", '').replace("\x14", '').replace("\x15", '').replace('/', '').replace('=', '').replace("〓", "") \
    .replace("»", "").replace("«", "").replace("¬", "").replace('`', '').replace (" -", "").replace("•", "")\
    .replace("l'", "").replace("l’", "").replace("l´", "").replace("d’", "").replace("d'", "").replace("d´","")\
    .replace("j’", "").replace("j'", "").replace("j´","").replace("n’", "").replace("n'", "").replace("n´","")\
    .replace("”", "").replace("~", "").replace("§", "").replace("¨", "").replace("©", "").replace("›", "")\
    .replace("₋", "").replace("→", "").replace("⇨", "").replace("∎", "").replace("√", "").replace("□", "")\
    .replace("*", "").replace("&", "").replace("►", "").replace("◊", "").replace("☞", "").replace("#", "")\
    .replace("%", "").replace("❖", "").replace("➠", "").replace("➢", "").replace("", "").replace("✓", "") \
    .replace("√", "").replace("✔", "").replace("♦", "").replace("◦", "").replace("●", "").replace("▫", "")\
    .replace("▪", "").replace("…", "").replace("þ", "").replace("®", "").replace('', '').replace("...", "")
    text = unidecode.unidecode(text) # remove accent
    return text

In [17]:
#On supprime les caractères étranges, accents et stop words
liste_cv_indeed_treated = [text_treatment(text) for text in liste_cv_indeed_fr]
#test
liste_cv_indeed_treated[0]

"informaticien developpement et reseaux  developpeur integrateur web  eragny  95   email me on indeed indeedcomrd7e8913ed00d0384  aujourhui je suis en recherche une opportunite sur un poste de developpeur ou integrateur web afin de monter toujours plus en competence et approfondir les bases solide que ai acquis en formation  experience  informaticien developpement et reseaux  legrandcercle95    eragny  95    novembre 2016  juin 2017  informaticien de entreprise mes missions etait de gerer les different probleme dans entreprise mise en place un antivirus serveur sauvegarde nas cree et gerer les droits sur active directory parametrer des clients leger ainsi que du materiel informatique comme des imprimantes ou des etiqueteuse en ip fixe  reajustement du code html et css sur le site grand public selon les normes w3c creation une source odbc creation un code en php  sql afin de recupere des donnees librairie sur un site fournisseur pour les enregistre en fiche xml creation de bannieres pou

** Gestion des stop words **

In [18]:
#generate stopwords
stop_words_py = set(stop_words.get_stop_words('french'))

# attention certains stop words pourraient être utiles par la suite
stopwords_set_manuel = set(["an", "ans", 'les', 'moins', 'd\'un','janvier', 'fevrier', 'février', 'mars', 'avril', \
                 'mai', 'juin', 'juillet', 'aout', 'août', 'septembre', 'octobre', 'novembre', 'décembre', \
                  'decembre', 'moins', 'mise', 'universit\xc3\xa9', 'université', 'universite', 'ion','sage', \
                  'o', 'rac', 'vers', 'via', 'p\xc3\xa9rim\xc3\xa8tre', 'périmètre','et','paris','x',"\x00",\
                          "\x01","\x02", "\x03","\x04","\x05","\x06","\x07","\x08","\x09","\x0e","\x0e","\x11",\
                           "\x12","\x13","\x14","\x15","\x16","\x17","\x18","\x19","transport","puis","lieu",\
                           "adresse","entre",'dun','dune','chez','boulognebillancourt','bt','etc','recrutement','main',\
                           'and', 'paie','paiement','environ','place','france','paris','mois','mobile','mobiles',\
                           'nanterre','source','sources','concerne','concernant','of','non','notes','rh','minimum',\
                           'maximum','bac','site','sites','actuellement','telephone','telephonique','telephoniques','ca','demenager',\
                           'demenagement','participer','participation','lycee','baccalaureat','lien','liens','in',\
                           'indeed','email','indeedcomrd7e8913ed00d0384','aujourhui','afin','toujours','enterprise',\
                           "guide","10g","11g","9i",'ad','v10','v2','v3','v5','v6','v8','v9','talan','talansolutions',\
                           "aide","ainsi",'aix','aupres','autour','autres','b','bonne','campagnes','cas','chaine',\
                           'choix','coherence','departement','differentes','differents','divers','fin','for','grandes',\
                           'i','ii','jour','lies','lors','lu','mettre','necessaires','national','nationale','nouvelle',\
                           'nouvelles','parle','partir','partie','permettant','permettte','plusieurs','reel','selon',\
                           'temps','toutes','v'])
stop_words_main = stop_words_py | stopwords_set_manuel
stop_words_main = list(stop_words_main)
print("taille stop words liste : ", len(stop_words_main))

taille stop words liste :  413


In [19]:
stop_words_main

['avril',
 'eussent',
 'seras',
 'été',
 'soyons',
 'fut',
 'octobre',
 'mettre',
 'devrons',
 'aurait',
 'août',
 'on',
 'devrait',
 'fûtes',
 'serez',
 'demenager',
 'valeur',
 'v3',
 'cela',
 'nommé',
 'pour',
 'dois',
 'ayez',
 'b',
 'o',
 'national',
 'dun',
 'force',
 'as',
 'lu',
 'ceux',
 'donc',
 'ai',
 'haut',
 'faisez',
 'votre',
 'minimum',
 'ni',
 'j',
 '\x0e',
 'avant',
 'son',
 'doit',
 'v9',
 '\x19',
 'ces',
 'là',
 '10g',
 'aurais',
 'ayons',
 'eussiez',
 'dù',
 'parle',
 'fussent',
 'sur',
 'v6',
 '\x15',
 'mai',
 'comment',
 'eut',
 'soit',
 'x',
 'font',
 'se',
 'differentes',
 'quelles',
 'est',
 'place',
 'elles',
 'qui',
 'v10',
 'sans',
 'te',
 'nanterre',
 'entre',
 'aurai',
 '\x16',
 'eux',
 'très',
 'depuis',
 'fin',
 'eût',
 'paris',
 '\x08',
 'quelle',
 'mot',
 'vont',
 'decembre',
 'fûmes',
 '\x02',
 'même',
 'lies',
 'and',
 'juste',
 'peut',
 'email',
 'universite',
 'nouveau',
 'que',
 'comme',
 'aide',
 'tels',
 'eus',
 'nationale',
 'telephonique',
 '

In [20]:
#voir si utile
def remove_stopwords(text,stopwords_list):
    text_temp = " ".join(text.split())+" "
    for word in stopwords_list:
        text_temp = text_temp.replace(" "+word+" ", " ")
    return text_temp

In [21]:
#test 
liste_cv_indeed_no_stop = [remove_stopwords(text,stop_words_main) for text in liste_cv_indeed_treated]
liste_cv_indeed_no_stop[0]

"informaticien developpement reseaux developpeur integrateur web eragny 95 recherche opportunite poste developpeur integrateur web monter plus competence approfondir bases solide acquis formation experience informaticien developpement reseaux legrandcercle95 eragny 95 2016 2017 informaticien entreprise missions etait gerer different probleme entreprise antivirus serveur sauvegarde nas cree gerer droits active directory parametrer clients leger materiel informatique imprimantes etiqueteuse ip fixe reajustement code html css grand public normes w3c creation odbc creation code php sql recupere donnees librairie fournisseur enregistre fiche xml creation bannieres evenements photoshop formation logiciel photoshop charge clientele europcar commercial saintouenaumone 95 2008 2016 missions principales qualite service assurer accueil clients respect charte agence gestion clients traiter ensemble appels analyser besoins client assurer suivi clientele logistique administratif s'assurer disponibil

In [22]:
#SnowballStemmer 
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("french")

In [23]:
def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems


def tokenize_only(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters 
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    return filtered_tokens

In [24]:
totalvocab_indeed_stemmed = []
totalvocab_indeed_tokenized = []
for text in liste_cv_indeed_no_stop:
    allwords_stemmed = tokenize_and_stem(text) #for each item in 'synopses', tokenize/stem
    totalvocab_indeed_stemmed.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(text)
    totalvocab_indeed_tokenized.extend(allwords_tokenized)

In [25]:
vocab_frame_indeed = pd.DataFrame({'words': totalvocab_indeed_tokenized}, index = totalvocab_indeed_stemmed)
print('there are ' + str(vocab_frame_indeed.shape[0]) + ' items in vocab_frame')
vocab_frame_indeed

there are 302629 items in vocab_frame


,words
informaticien,informaticien
developp,developpement
reseau,reseaux
developpeur,developpeur
integr,integrateur
web,web
eragny,eragny
recherch,recherche
opportunit,opportunite
post,poste


In [26]:
vocab_frame_indeed.loc['in']

words    ins
Name: in, dtype: object

### HAC

#### Bag of word tf-idf

In [27]:
tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=0.8,min_df=0.05,\
                           preprocessor=text_treatment,tokenizer=tokenize_and_stem)
bow_idf_indeed = tf_vect.fit_transform(liste_cv_indeed_no_stop)
l2_norm = make_pipeline(Normalizer(copy=False))
bow_idf_indeed = l2_norm.fit_transform(bow_idf_indeed)

vocab_liste_indeed = tf_vect.get_feature_names()

#Ajout une étape pour supprimer les doublons & shuffle
buffer = pd.DataFrame(data=bow_idf_indeed.toarray())
buffer = shuffle(buffer)
buffer.drop_duplicates(inplace=True)

new_idx = buffer.index
bow_idf_indeed = buffer.values
liste_files_new = []
for idx in new_idx:
    liste_files_new.append(liste_files_fr[idx])

In [28]:
hac = AgglomerativeClustering(n_clusters=5, affinity='euclidean', compute_full_tree='auto', linkage='ward')
hac.fit(bow_idf_indeed)

hac.labels_

array([1, 0, 3, ..., 1, 0, 1])

In [29]:
pd.DataFrame(hac.labels_, columns=["Label"]).groupby(["Label"])["Label"].size()

Label
0    552
1    351
2     74
3     86
4    130
Name: Label, dtype: int64

#### Find topics behind each label

In [30]:
#select randomly n doc idx in each cluster
#for each doc return n strongest tf-idf
dict_rand_word_label = dict()
for label in np.unique(hac.labels_):
    idx_label = np.where(hac.labels_==label)[0]
    idx_rand = np.random.choice(idx_label,size=(20))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf_indeed[idx])[::-1]
        for j in range(0, 4):  # nombre de motss
            text += vocab_frame_indeed.loc[vocab_liste_indeed[idx_ordered[j]]].values.tolist()[0][0] + " "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [32]:
dict_rand_word_label["cluster0"]

[' referentiel chargee courant suivi ',
 ' migrations audit oeuvre techniques ',
 ' configuration visual plateforme studio ',
 ' marketing animation responsabilites stage ',
 ' ecole commerce negociation telecom ',
 ' deploiements finance pays projets ',
 ' ibm developpement e administratif ',
 ' controle assistant commande gestion ',
 ' marketing data analyser strategie ',
 ' resultat optimisation organisations amelioration ',
 ' fournisseur industrie entreprise charge ',
 ' filiale suivi entites budget ',
 ' risques credit pilotage responsable ',
 ' publications affaires politique veille ',
 ' groupes politique management developpement ',
 ' entretien it profils selection ',
 ' centrale qualite activite ecole ',
 ' business assistant clients preparateur ',
 ' gestionnaires saisie personnel sociale ',
 ' definition business projets international ']

### ACP X HAC

Dimension reduction before applying HAC

In [33]:
# implement LSA
lsa_number = 170
svd = TruncatedSVD(lsa_number)
bow_idf_reduced_lsa = svd.fit_transform(bow_idf_indeed)
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

hac = AgglomerativeClustering(n_clusters=5, affinity='euclidean', compute_full_tree='auto', linkage='ward')
hac.fit(bow_idf_reduced_lsa)
labels_hac_lsa = hac.labels_

Explained variance of the SVD step: 77%


In [34]:
pd.DataFrame(hac.labels_, columns=["Label"]).groupby(["Label"])["Label"].size()

Label
0    386
1    381
2    134
3     46
4    246
Name: Label, dtype: int64

#### Find topic behind each cluster

In [35]:
dict_rand_word_label = dict()
for label in np.unique(labels_hac_lsa):
    idx_label = np.where(labels_hac_lsa==label)[0]
    idx_rand = np.random.choice(idx_label,size=(20))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf_indeed[idx])[::-1]
        for j in range(0, 4):  # nombre de motss
            text += vocab_frame_indeed.loc[vocab_liste_indeed[idx_ordered[j]]].values.tolist()[0][0] + " "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [36]:
dict_rand_word_label["cluster1"]

[' commercial presentation offres detecter ',
 ' moe personnel gestionnaires alternance ',
 ' europe responsable vente commercial ',
 ' humaines prospection ressources niveau ',
 ' alternance locaux marches expertise ',
 ' financiers audit specialisation banque ',
 ' clients plus clientele equipe ',
 ' marketing analytics performances digital ',
 ' new digital to media ',
 ' commercial business directeur partenariat ',
 ' controle comptabilite suivi comptable ',
 ' entretien crm suivi reporting ',
 ' filiale suivi entites budget ',
 ' marketing erp offres responsable ',
 ' gestionnaires finance structure ecommerce ',
 ' economie finance flux comptable ',
 ' comptabilite responsabilites fournisseur comptable ',
 ' innovantes bilan economie developpement ',
 ' achats complets definition principales ',
 ' marketing strategie direction marques ']

#### CV TALAN

In [37]:
#download talan CVs 
liste_cv_talan = []
liste_files_talan = []
path = '../Data_talan/txt/'
filenames = sorted(glob(os.path.join(path,"*.txt")))
print(len(filenames))
for file in filenames:
    liste_cv_talan.append(open(file).read())
    liste_files_talan.append(file.split('/')[-1].split('.')[0])

632


In [38]:
liste_cv_talan[0]

"Secteurs Télécom & média Activités métier Avant-vente, Développement Intégration & validation. Compétences fonctionnelles CRM : Elaboration de réponses aux appels d’offres, conception des processus métiers, rédaction des spécifications fonctionnelles, préparations des maquettes. Compétences techniques Microsoft Dynamics CRM 2011,2013 Salesforce CRM Architecture et plan de migration (volumétrie importante) Service Director : solution de QOS Oblicore Guarantee : solution SLM Développement : J2EE (EJB3, JMS), Webservice, API, Javascript,C#. Base de données : SQL Server, Oracle, PostgreSQL, MySQL Méthodologie Agile: Scrum, Extreem programing, Sure Step , UML. Atos France Bull France CRM Dynamics 2013, Salesforce CRM Dynamics 2011 Avant-vente CRM : Elaboration de réponses aux appels d’offres, conception des processus métiers, rédaction des spécifications fonctionnelles, préparations des maquettes SLM : Gestion des contrats SLM (Service Level Management) Prise de commande Télécom (SI) Proce

In [39]:
#suppression des saut de lignes
liste_cv_talan = [del_line_feed(text).lower() for text in liste_cv_talan]

In [40]:
#suppression de la ponctuation
liste_cv_talan_no_punc = [del_punct(text) for text in liste_cv_talan]

In [41]:
#selectionner seulement cvs fr
liste_cv_talan_fr, liste_files_talan_fr = get_cv_langue(liste_cv_talan_no_punc,'french',liste_files_talan)

nb_cv = len(liste_cv_talan_no_punc)
nb_cv_fr = len(liste_cv_talan_fr)

print("proportion cv french :",1- ((nb_cv-nb_cv_fr)/nb_cv))

proportion cv french : 0.9636075949367089


In [42]:
#On supprime les caractères étranges, accents et stop words
liste_cv_treated_talan = [text_treatment(text) for text in liste_cv_talan_fr]

In [43]:
#remove stop word
liste_cv_talan_no_stop = [remove_stopwords(text,stop_words_main) for text in liste_cv_treated_talan]

In [ ]:
#facultatif add only for talan cv (delete numbers) -> could be use for the preprocessing in general !
liste_cv_talan_clean = [re.sub('[0-9 ]+', ' ', text) for text in liste_cv_talan_no_stop]
liste_cv_talan_clean[2]

"competences sectorielles finance sante serveurs applications progiciels reuters powerplus pro methodologies xml langages outils developpement sql visual basic html materiel systemes exploitation windows nt bases donnees relationnelles oracle consultant junior formation ecole ingenieur ecole internationale sciences traitement information e s genie mathematique option ingenierie financiere competences experience personelle sejour linguistique famille ecossaise approfondissement anglais decouverte milieu equestre experience professionnelle stage pole bourse etrangere procapital - projet gestion execution operations financieres objet projet controle gestion execution transactions financieres controle trades etrangers depositaires marches rapprochement quotidien clients - marches etrangers collecte transactions passees fortis merrill lynch forme fichiers extraits ifac base donnees controle transactions passees fortis merrill lynch gestion anticipation achat vente cash devises extraction in

In [ ]:
totalvocab_stemmed_talan = []
totalvocab_tokenized_talan = []
for text in liste_cv_talan_no_stop:
    allwords_stemmed = tokenize_and_stem(text) #for each item in 'synopses', tokenize/stem
    totalvocab_stemmed_talan.extend(allwords_stemmed) #extend the 'totalvocab_stemmed' list
    allwords_tokenized = tokenize_only(text)
    totalvocab_tokenized_talan.extend(allwords_tokenized)

vocab_frame_talan = pd.DataFrame({'words': totalvocab_tokenized_talan}, index = totalvocab_stemmed_talan)
print('there are ' + str(vocab_frame_talan.shape[0]) + ' items in vocab_frame')
vocab_frame_talan

### HAC Talan

Representation bag of word tf-idf

In [ ]:
#TF IDF BOW Representation
tf_vect = TfidfVectorizer(stop_words=stop_words_main,max_df=0.7,min_df=0.1,\
                           preprocessor=text_treatment,tokenizer=tokenize_and_stem)
bow_idf_talan = tf_vect.fit_transform(liste_cv_talan_clean)


vocab_liste_talan = tf_vect.get_feature_names()

#Ajout une étape pour supprimer les doublons & shuffle
buffer = pd.DataFrame(data=bow_idf_talan.toarray())
buffer = shuffle(buffer)
buffer.drop_duplicates(inplace=True)

new_idx = buffer.index
bow_idf_talan = buffer.values
liste_files_new = []
liste_cv_clean_2 = []
for idx in new_idx:
    liste_files_new.append(liste_files_talan_fr[idx])
    liste_cv_clean_2.append(liste_cv_talan_clean[idx])

In [ ]:
hac = AgglomerativeClustering(n_clusters=5, affinity='euclidean', compute_full_tree='auto', linkage='ward')
hac.fit(bow_idf_talan)

pd.DataFrame(hac.labels_,columns=["Label"]).groupby(["Label"])["Label"].size()

In [ ]:
dict_rand_word_label = dict()
for label in np.unique(hac.labels_):
    idx_label = np.where(hac.labels_==label)[0]
    idx_rand = np.random.choice(idx_label,size=(12))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf_talan[idx])[::-1]
        for j in range(0, 4):  # nombre de motss
            text += vocab_frame_talan.loc[vocab_liste_talan[idx_ordered[j]]].values.tolist()[0][0] + " "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [ ]:
dict_rand_word_label["cluster1"]

#### ACP X HAC TALAN

In [ ]:
# implement LSA
lsa_number = 60
svd = TruncatedSVD(lsa_number)
bow_idf_reduced_talan = svd.fit_transform(bow_idf_talan)
explained_variance = svd.explained_variance_ratio_.sum()
print("Explained variance of the SVD step: {}%".format(int(explained_variance * 100)))

hac = AgglomerativeClustering(n_clusters=5, affinity='euclidean', compute_full_tree='auto', linkage='ward')
hac.fit(bow_idf_reduced_talan)
labels_hac_lsa = hac.labels_

In [ ]:
pd.DataFrame(labels_hac_lsa, columns=["Label"]).groupby(["Label"])["Label"].size()

In [ ]:
dict_rand_word_label = dict()
for label in np.unique(labels_hac_lsa):
    idx_label = np.where(labels_hac_lsa==label)[0]
    idx_rand = np.random.choice(idx_label,size=(12))
    liste_cluster_word = []
    for idx in idx_rand:
        text = " "
        idx_ordered = np.argsort(bow_idf_talan[idx])[::-1]
        for j in range(0, 4):  # nombre de motss
            text += vocab_frame_talan.loc[vocab_liste_talan[idx_ordered[j]]].values.tolist()[0][0] + " "
        liste_cluster_word.append(text)
    dict_rand_word_label["cluster"+str(label)] = liste_cluster_word

In [ ]:
dict_rand_word_label["cluster3"]

In [ ]:
idx_label = np.where(labels_hac_lsa==3)[0]
idx_label

In [ ]:
liste_cv_clean_2[99]